In [ ]:
import pickle
import pandas as pd
from collections import namedtuple

In [ ]:
from util_s2_gnn import make_dataset, roc_prc

In [ ]:
from s2_train_gnn_10 import Net, make_target, kmer_int

In [ ]:
import torch
import numpy as np

In [ ]:
import plotly.express as px

In [ ]:
with open('result/s2_gnn_run_10_10.pred_va_ep_99.pkl', 'rb') as f:
    data_debug = pickle.load(f)

In [ ]:
len(data_debug)

In [ ]:
auroc_all = []
auprc_all = []

for data in data_debug:
    auroc, auprc = roc_prc(torch.from_numpy(data['y']).float(), 
        torch.from_numpy(data['yp']).float(), 
        torch.from_numpy(data['m']).float())
    auroc_all.append(auroc)
    auprc_all.append(auprc)

In [ ]:
px.histogram(pd.DataFrame({'auROC': auroc_all}))

In [ ]:
px.histogram(pd.DataFrame({'auPRC': auprc_all}))

In [ ]:
px.scatter(pd.DataFrame({'auROC': auroc_all,
                         'auPRC': auprc_all}),
          x='auROC', y='auPRC')

In [ ]:
data = data_debug[0]

In [ ]:
roc_prc(torch.from_numpy(data['y']).float(), 
        torch.from_numpy(data['yp']).float(), 
        torch.from_numpy(data['m']).float())

In [ ]:
px.imshow(data['yp'] * data['m'])

In [ ]:
px.imshow(data['y'])

In [ ]:
model = Net([20, 20, 40, 40, 40, 50, 50, 50, 50, 100, 100, 100],
            k=3, embed_dim=50)
model.load_state_dict(torch.load('result/s2_gnn_run_10_10.model_ckpt_ep_99.pth', 
                                 map_location=torch.device('cpu')))
model.eval()

In [ ]:
TmpData = namedtuple("TmpData", ["x", "edge_index", "edge_attr"])

tmp_data = TmpData(x=torch.LongTensor(data['x']),
                  edge_index=torch.from_numpy(data['edge_index']).long(),
                  edge_attr=torch.from_numpy(data['edge_attr']).float())
tmp_pred = model(tmp_data)

In [ ]:
tmp_pred[:2, :2].detach().numpy()

In [ ]:
data['yp'][:2, :2]

In [ ]:
np.testing.assert_almost_equal(tmp_pred.detach().numpy(), data['yp'], decimal=5)

In [ ]:
roc_prc(torch.from_numpy(data['y']).float(), 
        torch.from_numpy(data['yp']).float(), 
        torch.from_numpy(data['m']).float())

In [ ]:
roc_prc(torch.from_numpy(data['y']).float(), 
        tmp_pred, 
        torch.from_numpy(data['m']).float())